In [1]:
import pandas as pd
import seaborn as sb
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score, KFold

import catboost as ctb

import eli5
from eli5.sklearn import PermutationImportance

In [8]:
df_sales_train = pd.read_hdf("sales_train.h5")
df_sales_test = pd.read_hdf("sales_test.h5")



df_sales_train["date"] = pd.to_datetime(df_sales_train["date"],dayfirst=True)

df_sales_train = df_sales_train.sort_values(by=["store", "dept", "date"])
df_sales_test["date"] = pd.to_datetime(df_sales_test["date"])

df_sales_train.head(10)

,id,weekly_sales,store,dept,date,is_holiday
134438,134438,24924.500000,1,1,2010-02-05,False
418402,418402,46039.488281,1,1,2010-02-12,True
115305,115305,41595.550781,1,1,2010-02-19,False
351187,351187,19403.539062,1,1,2010-02-26,False
91151,91151,21827.900391,1,1,2010-03-05,False
157905,157905,21043.390625,1,1,2010-03-12,False
72620,72620,22136.640625,1,1,2010-03-19,False
384329,384329,26229.210938,1,1,2010-03-26,False
81351,81351,57258.429688,1,1,2010-04-02,False
219935,219935,42960.910156,1,1,2010-04-09,False


In [3]:
df_features_train = pd.read_hdf("../input/features_train.h5")
df_features_test = pd.read_hdf("../input/features_test.h5")

In [4]:
df_stores_data = pd.read_csv("../input/stores_data.csv")
df_stores_data['Type_cat']=df_stores_data['Type'].factorize()[0]
df_stores_data.head()


,Store,Type,Size,Type_cat
0,1,A,151315,0
1,2,A,202307,0
2,3,B,37392,1
3,4,A,205863,0
4,5,B,34875,1


In [5]:
train_dict_Size = { x["Store"]: x["Size"] for x in df_stores_data.to_dict(orient="items").values() }
train_dict_Type = { x["Store"]: x["Type_cat"] for x in df_stores_data.to_dict(orient="items").values() }

In [6]:
df_sales_train["Type"] = df_sales_train.apply(lambda x: train_dict_Type.get( x["store"], -1), axis=1)
df_sales_test["Type"] = df_sales_test.apply(lambda x: train_dict_Type.get( x["store"], -1), axis=1)
df_sales_train["Surface"] = df_sales_train.apply(lambda x: train_dict_Size.get( x["store"], -1), axis=1)
df_sales_test["Surface"] = df_sales_test.apply(lambda x: train_dict_Size.get( x["store"], -1), axis=1)


In [7]:
df_sales_train

,id,weekly_sales,store,dept,date,is_holiday,Type,Surface
134438,134438,24924.500000,1,1,2010-02-05,False,0,151315
418402,418402,46039.488281,1,1,2010-02-12,True,0,151315
115305,115305,41595.550781,1,1,2010-02-19,False,0,151315
351187,351187,19403.539062,1,1,2010-02-26,False,0,151315
91151,91151,21827.900391,1,1,2010-03-05,False,0,151315
...,...,...,...,...,...,...,...,...
113867,113867,897.219971,45,98,2011-11-11,False,1,118221
356405,356405,503.200012,45,98,2011-11-18,False,1,118221
83977,83977,794.619995,45,98,2011-11-25,True,1,118221
164358,164358,538.510010,45,98,2011-12-02,False,1,118221


In [8]:
def cat_eng(df):
    factorized_values = df['is_holiday'].factorize()[0]
    df['is_holiday' + '_cat'] = factorized_values
    return df
df_sales_train = cat_eng(df_sales_train)
df_sales_test = cat_eng(df_sales_test)



In [9]:
def feature_engineering(df):
    df["dayofweek"] = df["date"].dt.dayofweek
    df["dayofyear"] = df["date"].dt.dayofyear
    df["week"] = df["date"].dt.week
    df["month"] = df["date"].dt.month

    
    return df

df_sales_train = feature_engineering(df_sales_train)
df_sales_test = feature_engineering(df_sales_test)

df_sales_train
df_dep = df_sales_train[ ["store", "dept",'month', "weekly_sales"] ].groupby(["store", "dept"]).agg(["mean", "std", "median", "size"])["weekly_sales"].reset_index()
df_dep.head()


df_sales_train_ext = pd.merge(df_sales_train, df_dep, on=["store", "dept"], how="left").fillna(-1)
df_sales_test_ext = pd.merge(df_sales_test, df_dep, on=["store", "dept"], how="left").fillna(-1)
df_sales_train_ext.head()

,id,weekly_sales,store,dept,date,is_holiday,Type,Surface,is_holiday_cat,dayofweek,dayofyear,week,month,mean,std,median,size
0,134438,24924.500000,1,1,2010-02-05,False,0,151315,0,4,36,5,2,22494.710938,9784.865234,18820.289062,97
1,418402,46039.488281,1,1,2010-02-12,True,0,151315,1,4,43,6,2,22494.710938,9784.865234,18820.289062,97
2,115305,41595.550781,1,1,2010-02-19,False,0,151315,0,4,50,7,2,22494.710938,9784.865234,18820.289062,97
3,351187,19403.539062,1,1,2010-02-26,False,0,151315,0,4,57,8,2,22494.710938,9784.865234,18820.289062,97
4,91151,21827.900391,1,1,2010-03-05,False,0,151315,0,4,64,9,3,22494.710938,9784.865234,18820.289062,97


In [10]:
df_features_train["date"] = pd.to_datetime(df_features_train["date"])

df_features_train= df_features_train.sort_values(by=["store", "date"])
df_features_test["date"] = pd.to_datetime(df_features_test["date"])


In [11]:
df_features_train = feature_engineering(df_features_train)
df_features_test = feature_engineering(df_features_test)

In [12]:
df_feat = df_features_train[ ["store",'month', "week",'fuel_price'] ].groupby(["store", "month"]).agg(["mean", "std", "median", "size"])["fuel_price"].reset_index()
df_feat = df_feat.rename(columns = {'size':'size_fuel','mean':'mean_fuel','std':'std_fuel', 'median':'median_fuel'})


In [13]:
df_sales_train_ext = pd.merge(df_sales_train_ext, df_feat, on=["store", "month"], how="left").fillna(-1)
df_sales_test_ext = pd.merge(df_sales_test_ext, df_feat, on=["store", "month"], how="left").fillna(-1)
df_sales_train_ext.head()

,id,weekly_sales,store,dept,date,is_holiday,Type,Surface,is_holiday_cat,dayofweek,...,week,month,mean,std,median,size,mean_fuel,std_fuel,median_fuel,size_fuel
0,134438,24924.500000,1,1,2010-02-05,False,0,151315,0,4,...,5,2,22494.710938,9784.865234,18820.289062,97,2.910156,0.353027,2.882812,8
1,418402,46039.488281,1,1,2010-02-12,True,0,151315,1,4,...,6,2,22494.710938,9784.865234,18820.289062,97,2.910156,0.353027,2.882812,8
2,115305,41595.550781,1,1,2010-02-19,False,0,151315,0,4,...,7,2,22494.710938,9784.865234,18820.289062,97,2.910156,0.353027,2.882812,8
3,351187,19403.539062,1,1,2010-02-26,False,0,151315,0,4,...,8,2,22494.710938,9784.865234,18820.289062,97,2.910156,0.353027,2.882812,8
4,91151,21827.900391,1,1,2010-03-05,False,0,151315,0,4,...,9,3,22494.710938,9784.865234,18820.289062,97,3.056641,0.473145,2.732422,7


In [14]:
df_feat_unemployment = df_features_train[ ["store",'month', "week",'unemployment'] ].groupby(["store", "month"]).agg(["mean", "std", "median", "size"])["unemployment"].reset_index()
df_feat_unemployment = df_feat_unemployment.rename(columns = {'size':'size_unemp','mean':'mean_unemp','std':'std_unemp', 'median':'median_unemp'})
df_feat_unemployment.head()

,store,month,mean_unemp,std_unemp,median_unemp,size_unemp
0,1,1,7.785156,0.099731,7.742188,6
1,1,2,7.890625,0.152588,7.835938,8
2,1,3,7.859375,0.177612,7.785156,7
3,1,4,7.765625,0.065796,7.773438,10
4,1,5,7.851562,0.161621,7.808594,10


In [15]:
df_feat_cpi = df_features_train[ ["store",'month', "week",'cpi'] ].groupby(["store", "month"]).agg(["mean", "std", "median", "size"])["cpi"].reset_index()
df_feat_cpi = df_feat_cpi.rename(columns = {'size':'size_cpi','mean':'mean_cpi','std':'std_cpi', 'median':'median_cpi'})
df_feat_cpi.head()

,store,month,mean_cpi,std_cpi,median_cpi,size_cpi
0,1,1,212.875,1.667969,212.000,6
1,1,2,213.250,2.771484,212.250,8
2,1,3,212.750,1.849609,211.625,7
3,1,4,213.375,2.681641,213.250,10
4,1,5,212.875,2.431641,211.750,10


In [16]:
df_sales_train_ext = pd.merge(df_sales_train_ext, df_feat_unemployment, on=["store", "month"], how="left").fillna(-1)
df_sales_train_ext = pd.merge(df_sales_train_ext, df_feat_cpi, on=["store", "month"], how="left").fillna(-1)
df_sales_test_ext = pd.merge(df_sales_test_ext, df_feat_unemployment, on=["store", "month"], how="left").fillna(-1)
df_sales_test_ext = pd.merge(df_sales_test_ext, df_feat_cpi, on=["store", "month"], how="left").fillna(-1)
df_sales_train_ext.head()

,id,weekly_sales,store,dept,date,is_holiday,Type,Surface,is_holiday_cat,dayofweek,...,median_fuel,size_fuel,mean_unemp,std_unemp,median_unemp,size_unemp,mean_cpi,std_cpi,median_cpi,size_cpi
0,134438,24924.500000,1,1,2010-02-05,False,0,151315,0,4,...,2.882812,8,7.890625,0.152588,7.835938,8,213.25,2.771484,212.250,8
1,418402,46039.488281,1,1,2010-02-12,True,0,151315,1,4,...,2.882812,8,7.890625,0.152588,7.835938,8,213.25,2.771484,212.250,8
2,115305,41595.550781,1,1,2010-02-19,False,0,151315,0,4,...,2.882812,8,7.890625,0.152588,7.835938,8,213.25,2.771484,212.250,8
3,351187,19403.539062,1,1,2010-02-26,False,0,151315,0,4,...,2.882812,8,7.890625,0.152588,7.835938,8,213.25,2.771484,212.250,8
4,91151,21827.900391,1,1,2010-03-05,False,0,151315,0,4,...,2.732422,7,7.859375,0.177612,7.785156,7,212.75,1.849609,211.625,7


In [17]:
df_sales_train_ext.columns

Index(['id', 'weekly_sales', 'store', 'dept', 'date', 'is_holiday', 'Type',
       'Surface', 'is_holiday_cat', 'dayofweek', 'dayofyear', 'week', 'month',
       'mean', 'std', 'median', 'size', 'mean_fuel', 'std_fuel', 'median_fuel',
       'size_fuel', 'mean_unemp', 'std_unemp', 'median_unemp', 'size_unemp',
       'mean_cpi', 'std_cpi', 'median_cpi', 'size_cpi'],
      dtype='object')

# Modeling

In [18]:
def run_model(model,feats):
  X = X_train
  y = np.log(y_train - np.min(y_train)+1)

  scores = cross_val_score(model, X, y, cv = 3, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [19]:
feats = ['store', 'dept', 'Type','size',
       'Surface', 'is_holiday_cat', 'dayofweek', 'dayofyear', 'week', 'month',
       'mean', 'std', 'median', 'mean_fuel', 'std_fuel', 'median_fuel',
        'mean_unemp', 'std_unemp', 'median_unemp',
       'mean_cpi', 'std_cpi', 'median_cpi']
#'median_fuel', 'mean_unemp', 'std_unemp', 'median_unemp','mean', 'std', 'median']
X_train = df_sales_train_ext[ feats ].values
X_test = df_sales_test_ext[ feats ].values

y_train = df_sales_train_ext["weekly_sales"].values
X_train = X_train[ y_train > 0 ]
y_train = y_train[ y_train > 0 ]
y_train_log = np.log(y_train - np.min(y_train) + 1)

In [ ]:
run_model(RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0),X_train)

In [105]:
['store'] -1.6845262588051062
['store','dept'] -1.3665726386200678
['store', 'dept','mean', ] -0.3259407877890872
['store', 'dept','mean', 'std'] -0.32680772718883894
['store', 'dept','mean', 'mean_fuel']-0.32572872014456866
['store', 'dept','mean', 'mean_fuel', 'is_holiday_cat']-0.32574981069408165
['store', 'dept','mean', 'mean_fuel', 'median'] -0.29658 98 8162532524
['store', 'dept','mean', 'mean_fuel', 'median','median_unemp']-0.29658 69 557374409 #6
['store', 'dept','mean', 'mean_fuel', 'median','mean_unemp']-0.29658 93 1266465977 -
['store','dept', 'mean', 'mean_fuel','median','Surface']-0.29658 74 372226265 #7
['store','dept', 'mean', 'mean_fuel','median','Type']-0.29658 36 5295854616 #4
['store','dept', 'mean', 'mean_fuel','median','week']-0.2965 697 9260228894#
['store','dept', 'mean', 'mean_fuel','median','dayofweek']-0.29658 79 378908123 #8
['store','dept', 'mean', 'mean_fuel','median','dayofyear']-0.2965 698 1875407884##
['store','dept', 'mean', 'mean_fuel','median','month'] -0.29658 56 511194634 #5 
['store','dept', 'mean', 'mean_fuel','median','std_fuel'] -0.29658 35 871532364 #4
['store','dept', 'mean', 'mean_fuel','median','median_fuel'] -0.29658 80 45683337 #9
['store','dept', 'mean', 'mean_fuel','median','std_unemp'] -0.2965 709 8034553675 ###
(-0.27960156803887765, 0.011946431920192516)

0.6758652661641265

In [21]:
xgb_params = {
    'max_depth': 9,
            'colsample_bytree': 0.9,
            'learning_rate': 0.15,
            'subsample': 0.9,
            'random_state': 21,
            'n_estimators':500,
}
#run_model(xgb.XGBRegressor(**xgb_params), feats)

In [ ]:
(-0.2542386551698049, 0.026945182575509978)

In [ ]:
ze wszystkimi -0.2652702232201894, 0.024873951696585468)
bez size_feat -0.2624167750279109

In [303]:
m = RandomForestRegressor(max_depth=10, n_estimators=100, random_state=0)
m.fit(X_train, y_train_log)
imp = PermutationImportance(m).fit(X_train, y_train_log)
eli5.show_weights(imp,feature_names = feats)

Weight,Feature
1.5605 ± 0.0057,dept
0.6038 ± 0.0061,Surface
0.2578 ± 0.0031,store
0.0486 ± 0.0008,Type
0.0111 ± 0.0001,dayofyear
0.0084 ± 0.0007,std_fuel
0.0066 ± 0.0002,week
0.0049 ± 0.0004,dayofweek
0.0038 ± 0.0001,std_unemp
0.0037 ± 0.0001,mean_unemp


In [22]:
model = xgb.XGBRegressor(**xgb_params)
model.fit(X_train, y_train_log)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.15, max_delta_step=0, max_depth=9,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=21,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.9,
             tree_method='exact', validate_parameters=1, verbosity=None)

# File

In [23]:
df_sales_test["weekly_sales"] = np.exp(model.predict(X_test))

df_sales_test[ ["id", "weekly_sales"] ].to_csv("../output/ML_last_try.csv", index=False)